In [1]:
import re
import string
import swifter
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('english')

In [2]:
df = pd.read_json('data/recipe1M_layers/layer1.json')[['ingredients']]

In [3]:
def preprocessor(text):
    text = text.split(',')[0]
    text = text.split(' or ')[0]
    text = text.split(' for ')[0]
    text = re.sub('[^a-zA-Z]\sto\s[^a-zA-Z]','',text)
    text = text.split(' to ')[0]

    text = text.lower()
    
    text = re.sub('(\((?:\(??[^\(]*?\)))','',text)
    # Removes quotation marks.
    text = text.replace('"', "")
    
    # Remove numeric characters.
    text = re.sub('\w*\d\w*', ' ', text)
    
    # Remove puncuation.
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    
    text = re.sub('\s+', ' ', text).strip()
    
    text = " ".join([stemmer.stem(x) for x in text.split(' ') if (x not in unit_words)])
    
#     text = stemmer.stem(text)
    
    text = text.title()
    
    text = text.replace(' ','')
    
    return text

unit_words = {'bushel',
              'cup',
              'dash',
              'drop',
              'fl. oz',
              'g',
              'gallon',
              'glass',
              'kg',
              'liter',
              'ml',
              'ounce',
              'pinch',
              'pint',
              'pound',
              'quart',
              'scoop',
              'shot',
              'tablespoon',
              'teaspoon',
 
              'bushels',
              'cups',
              'dashes',
              'drops',
              'fl. oz',
              'gram',
              'grams',
              'gallons',
              'glasses',
              'kg',
              'liters',
              'ml',
              'ounces',
              'pinches',
              'pints',
              'pounds',
              'quarts',
              'scoops',
              'shots',
              'tablespoons',
              'teaspoons',
              
              'clove',
              'cloves',
              'tsp',
              'tsps',
              'tbsp',
              'tbsps',
              'lb',
              'lbs',
              'c',
              'oz',
              'qt',
              'x',
              'sm',
              'to',
              'tbs',
              'lrg',
              'large',
              'medium',
              'small',
              
              'fresh',
              'freshly',
              
              'chopped',
              'finely',
              'coarsely',
              'coarse',
              'sifted',
              'mashed',
              'whole',
              'extra',
              'diced',
              'crumbled',
              'pasta',
              
              'thin',
              'thinly',
              'sliced',
              
              'grated',
              'shredded',
              'yolk',
              'white',
              'yolks',
              'whites',
              'all',
              'purpose'
}

In [4]:
preprocessor('egg White')

'Egg'

In [5]:
# df['instructions'] = df['instructions'].swifter.apply(lambda x: [y['text'] for y in x])
df['ingredients'] = df['ingredients'].swifter.apply(lambda x: [y['text'] for y in x])
# df['quantity'] = df['quantity'].apply(lambda x: [y['text'] for y in x])
# df['unit'] = df['unit'].apply(lambda x: [y['text'] for y in x])


In [6]:
df['ingredients'] = df['ingredients'].swifter.apply(lambda x: ([preprocessor(y) for y in x]))

In [7]:
def flatten(t):
    return [item for sublist in t for item in sublist]
from collections import Counter

counted = Counter(flatten(df['ingredients'].tolist()))
counted.most_common()

[('Salt', 354769),
 ('Egg', 262987),
 ('Sugar', 234742),
 ('Butter', 202515),
 ('Flour', 197660),
 ('Garlic', 197479),
 ('Onion', 185105),
 ('Water', 144365),
 ('OlivOil', 137419),
 ('Milk', 108957),
 ('LemonJuic', 79386),
 ('Pepper', 74182),
 ('', 68530),
 ('BakePowder', 67713),
 ('VegetOil', 64191),
 ('GroundBlackPepper', 61290),
 ('VanillaExtract', 60018),
 ('BakeSoda', 58626),
 ('UnsaltButter', 58088),
 ('SaltAndPepper', 57544),
 ('VirginOlivOil', 56957),
 ('BrownSugar', 56091),
 ('Carrot', 55737),
 ('Vanilla', 50839),
 ('BlackPepper', 47186),
 ('ParmesanChees', 46613),
 ('KosherSalt', 44411),
 ('Parsley', 44327),
 ('Cinnamon', 43919),
 ('Tomato', 41611),
 ('Honey', 40449),
 ('SoySauc', 38609),
 ('SourCream', 38545),
 ('GreenOnion', 37836),
 ('GarlicPowder', 36626),
 ('GranulSugar', 35011),
 ('GroundCinnamon', 34033),
 ('Oil', 33326),
 ('HeaviCream', 33281),
 ('Mayonnais', 32047),
 ('Lemon', 31510),
 ('RedOnion', 31272),
 ('Cornstarch', 30611),
 ('Cilantro', 29497),
 ('CheddarChees

In [17]:
import matplotlib.pyplot as plt
counted.most_common()[:12]


[('Salt', 354769),
 ('Egg', 262987),
 ('Sugar', 234742),
 ('Butter', 202515),
 ('Flour', 197660),
 ('Garlic', 197479),
 ('Onion', 185105),
 ('Water', 144365),
 ('OlivOil', 137419),
 ('Milk', 108957),
 ('LemonJuic', 79386),
 ('Pepper', 74182)]

In [8]:
df['ingredients'] = df['ingredients'].swifter.apply(lambda x: ' '.join(x))

In [9]:
df.iloc[2]['ingredients']


'Tomato KosherSalt RedOnion GreenBellPepper RedBellPepper YellowBellPepper Cucumb VirginOlivOil LeavBasil'

In [10]:
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(lowercase=False,min_df=2,max_df=0.8)
nmf_model = NMF(30)

doc_word = tfidf.fit_transform(df['ingredients'])
doc_topic = nmf_model.fit_transform(doc_word)

In [11]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
        
display_topics(nmf_model, tfidf.get_feature_names(), 5)



Topic  0
Flour, BakePowder, BakeSoda, Salt, Buttermilk

Topic  1
Salt, Pepper, Vinegar, Parsley, GroundBlackPepper

Topic  2
Sugar, Strawberri, Cornstarch, Vinegar, Salt

Topic  3
Butter, Flour, Pecan, Mushroom, ChickenBroth

Topic  4
VirginOlivOil, Garlic, GroundPepper, Shallot, BalsamVinegar

Topic  5
Milk, CheddarChees, BakePowder, Flour, Margarin

Topic  6
VanillaExtract, PowderSugar, ConfectionSugar, Pecan, BakeSoda

Topic  7
Onion, Garlic, GroundBeef, GreenPepper, CanTomato

Topic  8
BrownSugar, Ketchup, WorcestershirSauc, ChocolChip, Pecan

Topic  9
Water, Cornstarch, Salt, Vinegar, BreadFlour

Topic  10
Egg, Oil, CreamOfTartar, CakeFlour, Breadcrumb

Topic  11
OlivOil, Garlic, BalsamVinegar, Basil, Parsley

Topic  12
LemonJuic, LemonZest, Parsley, Mayonnais, WorcestershirSauc

Topic  13
UnsaltButter, PureVanillaExtract, ConfectionSugar, KosherSalt, LightBrownSugar

Topic  14
GarlicPowder, BlackPepper, Paprika, OnionPowder, CayennPepper

Topic  15
Cinnamon, Nutmeg, Appl, Raisin